# 目的
遺伝子学的分類に基づいた、再発の2値分類を実施する。  

In [1]:
# 基本的なライブラリ
import itertools
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

# models
from sklearn.linear_model import LogisticRegression  # ロジスティック回帰
from sklearn.neighbors import KNeighborsClassifier  # K近傍法
from sklearn.svm import SVC  # サポートベクターマシン
from sklearn.tree import DecisionTreeClassifier, export_graphviz  # 決定木
from sklearn.ensemble import RandomForestClassifier  # ランダムフォレスト
from sklearn.ensemble import AdaBoostClassifier  # AdaBoost
from sklearn.naive_bayes import GaussianNB  # ナイーブ・ベイズ
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA  # 二次判別分析
from sklearn.ensemble import ExtraTreesClassifier
from lightgbm import LGBMClassifier
import xgboost as xgb


# 評価指標
from sklearn.metrics import roc_auc_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


# local libraries
import config
import functions

SEED = config.SEED
functions.fix_seed(SEED)


# 最大表示列数の指定（ここでは50列を指定）N
pd.set_option("display.max_columns", 50)
pd.set_option("display.max_rows", 50)

%matplotlib inline

/home/breast-cancer-analytics/notebooks/functions.py:196: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  y_train: pd.Series(),
/home/breast-cancer-analytics/notebooks/functions.py:198: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  y_val: pd.Series(),


In [2]:
df_recurrenced = pd.read_pickle(
    config.INTERIM_PREPROCESSED_RECURRENCE_DIR + "/df_recurrenced.pkl"
)

classifiers = [
    LogisticRegression(max_iter=2000, random_state=SEED),
    KNeighborsClassifier(),
    SVC(
        kernel="linear",
        random_state=SEED,
        class_weight="balanced",
    ),
    SVC(kernel="poly", random_state=SEED, class_weight="balanced"),
    SVC(kernel="rbf", random_state=SEED, class_weight="balanced"),
    SVC(kernel="sigmoid", random_state=SEED, class_weight="balanced"),
    DecisionTreeClassifier(
        min_samples_split=20,
        min_samples_leaf=15,
        random_state=SEED,
        class_weight="balanced",
    ),
    RandomForestClassifier(
        min_samples_split=20,
        min_samples_leaf=15,
        random_state=SEED,
        class_weight="balanced",
    ),
    AdaBoostClassifier(random_state=SEED),
    GaussianNB(),
    GradientBoostingClassifier(random_state=SEED),
    SGDClassifier(random_state=SEED, class_weight="balanced"),
    QDA(),
    LGBMClassifier(class_weight="balanced", random_state=SEED),
    ExtraTreesClassifier(class_weight="balanced", random_state=SEED),
]

# データ全体での予測モデル検証

## ベースモデルの学習結果

各特徴量選択されたdfを用いて基本的な2値分類モデルを学習する

In [3]:
def validate_models(
    input_file_path: str = None,
    input_file_name: str = None,
    classifiers: list = config.classifiers,
    plot: bool = False,
):
    try:
        list_train, list_val, _, _ = functions.read_preprocessed_df(
            input_file_path, input_file_name
        )
    except:
        return
    X_train, y_train = list_train[0], list_train[1]
    X_val, y_val = list_val[0], list_val[1]
    print(X_train.shape, X_val.shape)
    display(
        functions.compare_bcms(
            X_train,
            y_train,
            X_val,
            y_val,
            classifiers=classifiers,
            plot=plot,
        )
    )


# サブグループ削除（一時的に）
subgroup_columns = [
    "CLAUDIN_SUBTYPE",
    # "NPI_CAT",
    # "TUMOR_CAT",
    # "LYMPH_CAT",
]

feature_selection_methods = [
    # "vt",
    # "mrmr"
    "none"
]

scaling_methods = [
    "std"
    # "norm"
]

dimensionality_reduction_methods = [
    "pca"
    # none
]

sampling_methods = [
     "none",
    #"smote"
]


for (
    feature_selection_method,
    scaling_method,
    dimensionality_reduction_method,
    sampling_method,
) in tqdm(
    itertools.product(
        feature_selection_methods,
        scaling_methods,
        dimensionality_reduction_methods,
        sampling_methods,
    )
):
    preprocess_order = "{0}_{1}_{2}_{3}".format(
        feature_selection_method,
        scaling_method,
        dimensionality_reduction_method,
        sampling_method,
    )
    print("=====" * 10)
    for subgroup_column in subgroup_columns:  # 各サブグループへの適用
        print("-----" * 10)
        print("subgroup column: ", subgroup_column)

        for subgroup in df_recurrenced[subgroup_column].unique():  # サブグループ毎への適用
            print("....." * 10)
            print("subgroup: ", subgroup)
            input_file_path = "./{0}/{1}/{2}".format(
                config.INTERIM_PREPROCESSED_RECURRENCE_DIR,
                subgroup_column,
                preprocess_order,
                plot=True,
            )
            print(input_file_path)
            validate_models(
                input_file_path=input_file_path, input_file_name=subgroup, plot=False
            )

0it [00:00, ?it/s]

--------------------------------------------------
subgroup column:  CLAUDIN_SUBTYPE
..................................................
subgroup:  LumB
./../data/interim/PREPROCESSED/RECURRENCE/CLAUDIN_SUBTYPE/none_std_pca_none
(178, 158) (20, 158)


,accuracy_score_train,accuracy_score_val,f1_score_train,f1_score_val,matthews_corrcoef_train,matthews_corrcoef_val,cohen_kappa_score_train,cohen_kappa_score_val,roc_auc_score_train,roc_auc_score_val
classifier,,,,,,,,,,
AdaBoostClassifier,1.000000,0.350,1.000000,0.434783,1.000000,-0.235702,1.000000,-0.181818,1.000000,0.395833
DecisionTreeClassifier,0.808989,0.500,0.721311,0.375000,0.595626,-0.041667,0.580770,-0.041667,0.777249,0.479167
ExtraTreesClassifier,1.000000,0.700,1.000000,0.571429,1.000000,0.356348,1.000000,0.347826,1.000000,0.666667
GaussianNB,0.904494,0.400,0.882759,0.571429,0.803711,0.000000,0.802351,0.000000,0.906217,0.500000
GradientBoostingClassifier,1.000000,0.600,1.000000,0.200000,1.000000,0.068041,1.000000,0.047619,1.000000,0.520833
KNeighborsClassifier,0.764045,0.650,0.671875,0.461538,0.495465,0.235702,0.490180,0.222222,0.737698,0.604167
LGBMClassifier,1.000000,0.650,1.000000,0.533333,1.000000,0.256776,1.000000,0.255319,1.000000,0.625000
LogisticRegression,1.000000,0.550,1.000000,0.307692,1.000000,0.000000,1.000000,0.000000,1.000000,0.500000
QuadraticDiscriminantAnalysis,1.000000,0.500,1.000000,0.500000,1.000000,0.041667,1.000000,0.038462,1.000000,0.520833


..................................................
subgroup:  Her2
./../data/interim/PREPROCESSED/RECURRENCE/CLAUDIN_SUBTYPE/none_std_pca_none
(90, 80) (11, 80)


,accuracy_score_train,accuracy_score_val,f1_score_train,f1_score_val,matthews_corrcoef_train,matthews_corrcoef_val,cohen_kappa_score_train,cohen_kappa_score_val,roc_auc_score_train,roc_auc_score_val
classifier,,,,,,,,,,
AdaBoostClassifier,1.000000,0.454545,1.000000,0.250000,1.000000,-0.149071,1.000000,-0.137931,1.000000,0.416667
DecisionTreeClassifier,0.766667,0.727273,0.655738,0.666667,0.530660,0.559017,0.494923,0.476190,0.800454,0.812500
ExtraTreesClassifier,1.000000,0.727273,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.500000
GaussianNB,0.988889,0.454545,0.977778,0.400000,0.970801,0.038576,0.970375,0.029412,0.978261,0.520833
GradientBoostingClassifier,1.000000,0.636364,1.000000,0.000000,1.000000,-0.193649,1.000000,-0.157895,1.000000,0.437500
KNeighborsClassifier,0.788889,0.909091,0.536585,0.800000,0.407585,0.769800,0.402516,0.744186,0.686892,0.833333
LGBMClassifier,1.000000,0.636364,1.000000,0.333333,1.000000,0.083333,1.000000,0.083333,1.000000,0.541667
LogisticRegression,1.000000,0.818182,1.000000,0.500000,1.000000,0.516398,1.000000,0.421053,1.000000,0.666667
QuadraticDiscriminantAnalysis,1.000000,0.636364,1.000000,0.333333,1.000000,0.083333,1.000000,0.083333,1.000000,0.541667


..................................................
subgroup:  LumA
./../data/interim/PREPROCESSED/RECURRENCE/CLAUDIN_SUBTYPE/none_std_pca_none
(178, 158) (20, 158)


,accuracy_score_train,accuracy_score_val,f1_score_train,f1_score_val,matthews_corrcoef_train,matthews_corrcoef_val,cohen_kappa_score_train,cohen_kappa_score_val,roc_auc_score_train,roc_auc_score_val
classifier,,,,,,,,,,
AdaBoostClassifier,1.000000,0.75,1.000000,0.761905,1.000000,0.502519,1.000000,0.500000,1.000000,0.752525
DecisionTreeClassifier,0.747191,0.65,0.736842,0.666667,0.514737,0.301511,0.500561,0.300000,0.754954,0.651515
ExtraTreesClassifier,1.000000,0.70,1.000000,0.750000,1.000000,0.389819,1.000000,0.381443,1.000000,0.686869
GaussianNB,0.882022,0.55,0.894472,0.689655,0.763514,0.033501,0.761089,0.021739,0.878176,0.510101
GradientBoostingClassifier,1.000000,0.60,1.000000,0.636364,1.000000,0.191919,1.000000,0.191919,1.000000,0.595960
KNeighborsClassifier,0.657303,0.55,0.666667,0.400000,0.317437,0.201008,0.315816,0.150943,0.659172,0.580808
LGBMClassifier,1.000000,0.75,1.000000,0.736842,1.000000,0.533396,1.000000,0.509804,1.000000,0.762626
LogisticRegression,1.000000,0.60,1.000000,0.692308,1.000000,0.174078,1.000000,0.157895,1.000000,0.575758
QuadraticDiscriminantAnalysis,1.000000,0.75,1.000000,0.800000,1.000000,0.504430,1.000000,0.479167,1.000000,0.732323


1it [00:08,  8.74s/it]

..................................................
subgroup:  claudin-low
./../data/interim/PREPROCESSED/RECURRENCE/CLAUDIN_SUBTYPE/none_std_pca_none
pkl file does not exist
..................................................
subgroup:  Basal
./../data/interim/PREPROCESSED/RECURRENCE/CLAUDIN_SUBTYPE/none_std_pca_none
pkl file does not exist
..................................................
subgroup:  Normal
./../data/interim/PREPROCESSED/RECURRENCE/CLAUDIN_SUBTYPE/none_std_pca_none
pkl file does not exist
..................................................
subgroup:  NC
./../data/interim/PREPROCESSED/RECURRENCE/CLAUDIN_SUBTYPE/none_std_pca_none
pkl file does not exist


## xgboost

In [4]:
def validate_models(
    input_file_path: str = None,
    input_file_name: str = None,
    classifiers: list = config.classifiers,
    plot: bool = False,
):
    try:
        list_train, list_val, _, _ = functions.read_preprocessed_df(
            input_file_path, input_file_name
        )
    except:
        return
    X_train, y_train = list_train[0], list_train[1]
    X_val, y_val = list_val[0], list_val[1]
    print(X_train.shape, X_val.shape)
    display(
        functions.compare_bcms(
            X_train,
            y_train,
            X_val,
            y_val,
            classifiers=classifiers,
            plot=plot,
        )
    )


for feature_selection_method, scaling_method, sampling_method in tqdm(
    itertools.product(feature_selection_methods, scaling_methods, sampling_methods)
):
    preprocess_order = "{0}_{1}_{2}".format(
        feature_selection_method, scaling_method, sampling_method
    )
    print("=====" * 10)
    for subgroup_column in subgroup_columns:  # 各サブグループへの適用
        print("-----" * 10)
        print("subgroup column: ", subgroup_column)

        for subgroup in df_recurrenced[subgroup_column].unique():  # サブグループ毎への適用
            print("....." * 10)
            print("subgroup: ", subgroup)
            input_file_path = "./{0}/{1}/{2}".format(
                config.INTERIM_PREPROCESSED_RECURRENCE_DIR,
                subgroup_column,
                preprocess_order,
                plot=True,
            )
            print(input_file_path)

            try:
                list_train, list_val, _, _ = functions.read_preprocessed_df(
                    input_file_path, subgroup
                )
            except:
                pass

            X_train, y_train = list_train[0], list_train[1]
            X_val, y_val = list_val[0], list_val[1]
            print(X_train.shape, X_val.shape)

            xgb_train = xgb.DMatrix(
                X_train, label=y_train, feature_names=X_train.columns
            )
            xgb_val = xgb.DMatrix(X_val, label=y_val, feature_names=X_val.columns)

            param = {
                # 二値分類問題
                "objective": "binary:logistic",
            }
            clf = xgb.train(param, xgb_train)
            y_val_pred_proba = clf.predict(xgb_val)
            y_val_pred = np.where(y_val_pred_proba > 0.5, 1, 0)
            functions.show_scores(y_val, y_val_pred)

0it [00:00, ?it/s]

--------------------------------------------------
subgroup column:  CLAUDIN_SUBTYPE
..................................................
subgroup:  LumB
./../data/interim/PREPROCESSED/RECURRENCE/CLAUDIN_SUBTYPE/none_std_none
pkl file does not exist


NameError: name 'list_train' is not defined